# Actividad 1.2: Raíces de polinomios
Cristobal Medina Meza | A01643121

In [15]:
import sys
sys.path.append(r'D:\codigos_6to\optimizacion\Actividad-1.2-Raices-de-polinomios\backend')
from MA2008B import *

Utilizando el método de Newton-Raphson, seguido del procedimiento de deflación, aproxime todas las raíces del siguiente polinomio con un máximo error de $10^{-3}$:

\begin{equation}
    p(x) = x^7 + 16.3x^6 + 91.31x^5 + 233.329x^4 + 294.662x^3 + 184.72276x^2 + 53.15896x + 5.25504.
\end{equation}


In [16]:
import numpy as np
from MA2008B import horner, deflacion_x_x0, newton_raphson

def p(x):
    return x**7 + 16.3*x**6 + 91.31*x**5 + 233.329*x**4 + 294.662*x**3 + 184.72276*x**2 + 53.15896*x + 5.25504

def dp(x):
    return 7*x**6 + 6*16.3*x**5 + 5*91.31*x**4 + 4*233.329*x**3 + 3*294.662*x**2 + 2*184.72276*x + 53.15896

tol = 1e-3
nmax = 100
roots = []

a = [1, 16.3, 91.31, 233.329, 294.662, 184.72276, 53.15896, 5.25504]

while len(a) > 1:
    x0 = np.random.rand() * 10 - 5
    root = newton_raphson(p, dp, x0, tol, nmax)
    
    if isinstance(root, str):
        print(root)
        break
    
    roots.append(root)
    
    a, _ = deflacion_x_x0(a, root)

    def p(x):
        return sum(coef * x**i for i, coef in enumerate(reversed(a)))

    def dp(x):
        return sum(i * coef * x**(i-1) for i, coef in enumerate(reversed(a)) if i > 0)

print("Raíces encontradas:", roots)

Raíces encontradas: [-0.19999999920959097, -0.49999987339767415, -0.6999997552069069, -1.2000007361897642, -2.299999454694993, -3.400000181761377, -7.9999999995396935]


Utilizando el método de Müller, aproxime las raíces reales y complejas de los siguientes polinomios con un máximo error de $10^{-3}$. Recuerde que si una raíz es compleja, su conjugado también es una raíz. En estos casos se puede usar deflación cuadrática para reducir el polinomio.

\begin{equation} p(x) = x^3 - x^2 + 2x - 2 \end{equation}



\begin{equation} p(x) = x^4 + 3x^2 + 5 \end{equation}


\begin{equation} p(x) = x^4 - 2x^3 + 6x^2 - 8x + 8 \end{equation}

In [17]:
import cmath
from MA2008B import muller, deflacion_x2_bc

def polynomial_1(x):
    return x**3 - x**2 + 2*x - 2

def polynomial_2(x):
    return x**4 + 3*x**2 + 5

def polynomial_3(x):
    return x**4 - 2*x**3 + 6*x**2 - 8*x + 8

def find_roots(f, initial_guesses, tol=1e-3, nmax=100):
    roots = []
    for i in range(len(initial_guesses) - 2):
        x0, x1, x2 = initial_guesses[i], initial_guesses[i+1], initial_guesses[i+2]
        root = muller(f, x0, x1, x2, tol, nmax)
        if isinstance(root, str):
            print(root)
            continue
        roots.append(root)
        if cmath.isclose(root.imag, 0, abs_tol=tol):
            f = lambda x: f(x) / (x - root)
        else:
            f = lambda x: deflacion_x2_bc(f, root.real, root.imag)[0]
    return roots

initial_guesses_1 = [0, 0.5, 1]
initial_guesses_2 = [0, 1, 2]
initial_guesses_3 = [0, 1, 2]

roots_1 = find_roots(polynomial_1, initial_guesses_1)
roots_2 = find_roots(polynomial_2, initial_guesses_2)
roots_3 = find_roots(polynomial_3, initial_guesses_3)

print("Roots of polynomial 1:", roots_1)
print("Roots of polynomial 2:", roots_2)
print("Roots of polynomial 3:", roots_3)

Roots of polynomial 1: [(1+0j)]
Roots of polynomial 2: [(0.6066580459058742-1.3667603947880538j)]
Roots of polynomial 3: [(1.000000428560886-1.0000006989980468j)]


In [21]:
import cmath
import numpy as np
from MA2008B import muller

def polynomial_1(x):
    return x**3 - x**2 + 2*x - 2

def polynomial_2(x):
    return x**4 + 3*x**2 + 5

def polynomial_3(x):
    return x**4 - 2*x**3 + 6*x**2 - 8*x + 8

class DeflatedPolynomial:
    """Class to represent a polynomial that is being deflated as roots are found"""
    
    def __init__(self, f):
        """Initialize with the original polynomial function"""
        self.original_f = f
        self.roots = []
        
    def __call__(self, x):
        """Evaluate the deflated polynomial at x"""
        # Start with the original polynomial value
        result = self.original_f(x)
        
        # Divide by (x - root) for each found root
        for root in self.roots:
            # Avoid division by zero or very small denominators
            if abs(x - root) < 1e-10:
                return 0
            result = result / (x - root)
            
        return result
    
    def add_root(self, root):
        """Add a root to the list of found roots"""
        self.roots.append(root)

def find_all_roots(f, degree, tol=1e-3, nmax=100):
    """
    Find all roots of a polynomial using Müller's method with deflation.
    """
    # Define various starting points to try
    starting_points = [
        [-2, -1, 0], [0, 1, 2], [-1, 0, 1],
        [0, 0.5, 1], [-1, -0.5, 0], [1j, 2j, 3j]
    ]
    
    deflated_poly = DeflatedPolynomial(f)
    roots = []
    
    while len(roots) < degree:
        found_root = False
        
        # Try different starting points
        for points in starting_points:
            try:
                root = muller(deflated_poly, points[0], points[1], points[2], tol, nmax)
                if not isinstance(root, str):
                    root = complex(root)
                    # Check if this root is close to any we've already found
                    if not any(abs(root - r) < tol for r in roots):
                        roots.append(root)
                        deflated_poly.add_root(root)
                        found_root = True
                        break
            except:
                continue
        
        # If we've tried all starting points but found no new root, break
        if not found_root:
            break
        
        # If we found a complex root with significant imaginary part, add its conjugate
        if abs(root.imag) > tol and len(roots) < degree:
            conjugate = complex(root.real, -root.imag)
            roots.append(conjugate)
            deflated_poly.add_root(conjugate)
    
    return roots

# Find and display roots
print("Roots of polynomial 1:")
roots_1 = find_all_roots(polynomial_1, 3)
for r in roots_1:
    if abs(r.imag) < 1e-3:
        print(f"  {r.real:.6f}")
    else:
        print(f"  {r.real:.6f} + {r.imag:.6f}j")

print("\nRoots of polynomial 2:")
roots_2 = find_all_roots(polynomial_2, 4)
for r in roots_2:
    if abs(r.imag) < 1e-3:
        print(f"  {r.real:.6f}")
    else:
        print(f"  {r.real:.6f} + {r.imag:.6f}j")

print("\nRoots of polynomial 3:")
roots_3 = find_all_roots(polynomial_3, 4)
for r in roots_3:
    if abs(r.imag) < 1e-3:
        print(f"  {r.real:.6f}")
    else:
        print(f"  {r.real:.6f} + {r.imag:.6f}j")

Roots of polynomial 1:
  0.000000 + 1.414214j
  0.000000 + -1.414214j
  1.000000

Roots of polynomial 2:
  -0.606658 + -1.366760j
  -0.606658 + 1.366760j
  0.606658 + -1.366760j
  0.606658 + 1.366760j

Roots of polynomial 3:
  1.000001 + -0.999999j
  1.000001 + 0.999999j
  0.000000 + -2.000000j
  0.000000 + 2.000000j


Use un sistema de computacion algebraica o librerıa de Python para verificar que los resultados de los problemas anteriores estan dentro del margen de error establecido

1.

In [18]:
import numpy as np

coeficientes = [1, 16.3, 91.31, 233.329, 294.662, 184.72276, 53.15896, 5.25504]
raices_exactas = np.roots(coeficientes)

print("\nRaíces exactas del polinomio (comprobadas con numpy):")
for raiz in raices_exactas:
    print(f"Raíz exacta: {raiz}")



Raíces exactas del polinomio (comprobadas con numpy):
Raíz exacta: -8.0
Raíz exacta: -3.4000000000000514
Raíz exacta: -2.2999999999999363
Raíz exacta: -1.2000000000000282
Raíz exacta: -0.6999999999999841
Raíz exacta: -0.5000000000000044
Raíz exacta: -0.20000000000000043


Resultados anteriores:
[-2.300000000085624, -1.199999999483634, -0.1999998839356626, -0.5000002621915055, -3.3999999953085127, -0.6999998370883357, -8.000000021906725]

2.

In [19]:
coefficients = [[1, -1, 2, -2], [1, 0, 3, 0, 5], [1, -2, 6, -8, 8]]

for l in range(len(coefficients)):
    roots = np.roots(coefficients[l])
    print(f"Roots for coefficient {l}:")
    for root in roots:
        print(root)


Roots for coefficient 0:
(5.828670879282072e-16+1.4142135623730943j)
(5.828670879282072e-16-1.4142135623730943j)
(1+0j)
Roots for coefficient 1:
(-0.6066580492747915+1.366760399173863j)
(-0.6066580492747915-1.366760399173863j)
(0.6066580492747911+1.3667603991738626j)
(0.6066580492747911-1.3667603991738626j)
Roots for coefficient 2:
(-2.220446049250313e-16+1.999999999999998j)
(-2.220446049250313e-16-1.999999999999998j)
(1+0.9999999999999996j)
(1-0.9999999999999996j)


Resultados anteriores:

Roots of polynomial 1:
  0.000000 + 1.414214j
  0.000000 + -1.414214j
  1.000000

Roots of polynomial 2:
  -0.606658 + -1.366760j
  -0.606658 + 1.366760j
  0.606658 + -1.366760j
  0.606658 + 1.366760j

Roots of polynomial 3:
  1.000001 + -0.999999j
  1.000001 + 0.999999j
  0.000000 + -2.000000j
  0.000000 + 2.000000j